### Data Preprocessing

In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [2]:
feat = pd.read_csv('data/emb_data4.csv')
feat = shuffle(feat)

In [3]:
X1 = feat['master_id']
X2 = feat.drop(['master_id', 'assigne_state'], axis=1)
X2 = scale(X2, axis=0)
y = feat['assigne_state']

In [4]:
u_masters = []

for i in range(len(X1)):
    if X1[i] not in u_masters:
        u_masters.append(X1[i])

In [5]:
from tqdm import tqdm
X_new = pd.DataFrame(columns=['master_idx'], index = X1.index)
for i in tqdm(range(len(X1))):
    X_new['master_idx'][i] = u_masters.index(X1[i])
        
print(X_new.shape)

100%|██████████| 113184/113184 [02:13<00:00, 849.89it/s]

(113184, 1)


In [12]:
tst_X_new = X_new[0:20000].as_matrix()
tst_X2 = X2[0:20000]
tst_y = y[0:20000]

val_X_new = X_new[20001:50000].as_matrix()
val_X2 = X2[20001:50000]
val_y = y[20001:50000]

trn_X_new = X_new[50001:].as_matrix()
trn_X2 = X2[50001:]
trn_y = y[50001:]

In [7]:
trn_X_new, val_X_new, trn_X2, val_X2, trn_y, val_y = train_test_split(X_new, X2, y, test_size=0.2, random_state=42)

### Parallel Model

In [13]:
from keras.layers import Input, Embedding, merge
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers.merge import concatenate
n_masters = X1.nunique()
n_factors = 40

In [18]:
master_in = Input(shape=(1,), dtype='int64', name='master_in')
m1 = Embedding(n_masters, n_factors, input_length=1)(master_in)
m2 = Flatten()(m1)
order_in = Input(shape=(12,), dtype='float32', name='order_in')

In [19]:
#x =merge([m2, order_in], mode = 'concat')
x = concatenate([m2, order_in])
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(200, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(100, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(1, activation='sigmoid')(x)

In [20]:
model = Model([master_in, order_in], x)
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
master_in (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_3 (Embedding)          (None, 1, 40)         117920      master_in[0][0]                  
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 40)            0           embedding_3[0][0]                
____________________________________________________________________________________________________
order_in (InputLayer)            (None, 12)            0                                            
___________________________________________________________________________________________

In [21]:
#model.optimizer.lr = 0.01

In [26]:
model.fit([trn_X_new, trn_X2], trn_y, batch_size = 1024, epochs=1000,
          validation_data=([val_X_new, val_X2], val_y),
          callbacks = [EarlyStopping(monitor='val_acc', patience=30, verbose=1, min_delta=1e-4, mode='max'),
                           #ReduceLROnPlateau(monitor='val_acc', factor=0.3, patience=5, verbose=1, epsilon=1e-4, mode='max')
                            CSVLogger('data/learning_log.csv', separator=',', append=False),
                            ReduceLROnPlateau(monitor='val_acc', factor=0.3, patience=8, min_lr=0.00001, verbose = 1)
                           ])

Train on 63183 samples, validate on 29999 samples
Epoch 1/1000
63183/63183 [==============================] - 3s - loss: 0.4052 - acc: 0.8161 - val_loss: 0.4339 - val_acc: 0.8054
Epoch 2/1000
63183/63183 [==============================] - 3s - loss: 0.4054 - acc: 0.8155 - val_loss: 0.4339 - val_acc: 0.8055
Epoch 3/1000
63183/63183 [==============================] - 3s - loss: 0.4042 - acc: 0.8162 - val_loss: 0.4339 - val_acc: 0.8054
Epoch 4/1000
63183/63183 [==============================] - 3s - loss: 0.4044 - acc: 0.8176 - val_loss: 0.4339 - val_acc: 0.8055
Epoch 5/1000
63183/63183 [==============================] - 3s - loss: 0.4060 - acc: 0.8163 - val_loss: 0.4339 - val_acc: 0.8055
Epoch 6/1000
63183/63183 [==============================] - 3s - loss: 0.4055 - acc: 0.8172 - val_loss: 0.4339 - val_acc: 0.8054
Epoch 7/1000
63183/63183 [==============================] - 3s - loss: 0.4042 - acc: 0.8178 - val_loss: 0.4339 - val_acc: 0.8056
Epoch 8/1000
63183/63183 [=====================

In [27]:
y_pred = model.predict([tst_X_new, tst_X2])

In [28]:
from sklearn.metrics import roc_auc_score    

In [29]:
roc_auc_score(tst_y, y_pred)

0.76589642311042894

### Bins

In [39]:
y_pred2 = y_pred.reshape((20000,))

In [40]:
bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
group_names = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

categories = pd.cut(y_pred2, bins, labels=group_names)

df = pd.DataFrame(columns=['y_true', 'y_pred', 'bin'])
df['y_true'] = tst_y
df['y_pred'] = y_pred2
df['bin'] = categories

df.to_csv('bins.csv')

### Sequential Model

In [ ]:
n_masters = X.nunique()
n_factors = 100

In [ ]:
from keras.layers import Input, Embedding
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.models import Sequential, Model

In [ ]:
emb_model = Sequential([
    Embedding(n_masters, n_factors, input_length=1),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [ ]:
emb_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])
emb_model.summary()

In [ ]:
emb_model.fit(X_new['master_idx'], y, batch_size=256, epochs=100)